# Segunda Avaliação Parcial

Neste módulo trabalhamos a visualização de dados, assim como as técnicas para acessar estes dados da internet. As técnicas de web scraping que estudamos, usando __Requests__, __Beautiful Soup__ e __Selenium__, permitem implementar aplicações que processam informação diretamente da rede.  

Desde a Instrução prática 7, por exemplo, estamos trabalhando com dados que baixamos do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos).  

Aprendemos também como implementar uma aplicação com GUI utilizando o __TK Inter__. 

Neste contexto vamos implementar uma aplicação que, a traves de uma GUI, permita gerar gráficos de precipitações e temperaturas medias mensais de um determinado ano, para uma estação meteorológica. A atividade deve ser organizada da seguinte forma.

## Exercício 1

__Primeiro protótipo__: Desenvolva uma interface gráfica que pegue, em tempo real, as informações sobre os dados de quais anos estão disponíveis no site do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). Os anos disponíveis devem ser apresentados utilizando um widget apropriado que permita selecionar um, e apenas um ano. Se não tiver conexão a Internet ou se o site não estiver disponível o aplicativo deve mostrar uma mensagem de erro na interface.


In [ ]:
# fazendo a requisicao
link = 'https://portal.inmet.gov.br/dadoshistoricos'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

try:
    requisicao = requests.get(link, headers=headers)
except ConnectionError:
    requisicao = None
    resposta = None
    raise Exception('Requisição não foi bem sucedida')
except Exception as e:
    requisicao = None
    resposta = None
    raise e
if requisicao:
    resposta = bs4.BeautifulSoup(requisicao.text, 'html.parser')
else:
    resposta = None

In [ ]:
# pegando anos e links
if resposta:
    lista_anos = {}
    lita_links = resposta.find('div', attrs={'id': 'main'}).find_all('a')
    for item in lita_links:
        link = item.attrs['href']
        ano = link.split('/')[-1]
        lista_anos[ano[:-4]] =  link 
    #return lista_anos
else:
    raise Exception('Requisição não foi bem sucedida')

In [ ]:
# Widget combobox
frameEstacao.inpAnos = Combobox(values=[], state='readonly')

anos = get_anos() # retorna um dicionario com os anos e links
anosList = list(anos.keys())
anosList.insert(0, '')
frameEstacao.inpAnos['values'] = anosList

## Exercício 2

__Segundo protótipo__: Com base no ano escolhido sua aplicação deve baixar o arquivo compactado, com os dados de todas as estações, descompactar ele numa pasta temporária e mostrar em um outro widget a lista de estações disponíveis para selecionar uma e apenas uma delas. O widget do protótipo anterior deve ficar disponível para que o usuário possa trocar o ano se desejar. Nesse caso um novo arquivo será baixado e descompactado e as estações disponíveis exibidas para seleção.

In [ ]:
# baixar e descompactar zip
def carregar_estacoes(self, link_ano: str) -> None:
    nome = link_ano.split('/')[-1]
    requisicao = requests.get(link_ano, headers=self.headers)
    
    if requisicao.status_code == 200:
        with open(link_ano.split('/')[-1], 'wb') as f:
            f.write(requisicao.content)
    else:
        raise Exception('Erro ao salvar o arquivo')
    
    with zipfile.ZipFile(nome, 'r') as zip_ref:
        self.pasta_ano = nome[:-4]
        zip_ref.extractall(self.pasta_ano)
    
    os.remove(nome)

In [ ]:
# pegar lista de estacoes de um ano
def get_estacoes(self) -> dict:
    estacoes={}

    if not os.path.exists(self.pasta_ano):
        raise Exception('Pasta não existe')

    for root, dirs, files in os.walk(f'{self.pasta_ano}'):
        for file in files:
            if file.endswith('.CSV') or file.endswith('.csv'):
                estacoes[file.split('_')[4]] = os.path.join(root, file)
    
    return estacoes

In [ ]:
# colocando na combobox
frameEstacao.inpEstacoes = Combobox(values=[], state='readonly')

estacaoes = get_estacoes() # retorna um dicionario com os anos e links
estacaoesList = list(anos.keys())
frameEstacao.inpEstacoes['values'] = estacaoesList

# Exercício 3

__Terceiro protótipo__: Com base na estação selecionada gere e exiba dentro da aplicação os gráficos de precipitações e temperaturas medias mensais para a estação selecionada. A interface deve permitir que o usuário troque a estação selecionada a qualquer momento, assim como foi feito co o ano, gerando novos gráficos.   

In [ ]:
df_tratado = plot.trata_dados(caminho_arquivo)

In [ ]:
# criar grafico
def get_plot(caminho_arquivo: str) -> plt.Figure:
    df = trata_dados(caminho_arquivo)
    
    fig, ax = plt.subplots(2, 1, figsize=(12, 6))
    
    acumulado_precipitacao = df.resample('ME')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].sum()
    media_temperatura = df.groupby(df.index.month)['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'].mean()
    meses = df.index.month.unique()
    
    dict_meses = {1: 'Jan', 2: 'Fev', 3: 'Mar', 
                  4: 'Abr', 5: 'Mai', 6: 'Jun', 
                  7: 'Jul', 8: 'Ago', 9: 'Set', 
                  10: 'Out', 11: 'Nov', 12: 'Dez'}
    
    meses = [dict_meses[mes] for mes in meses]
    
    ax[0].plot(meses, acumulado_precipitacao, marker='o')
    ax[0].set_title('Acumulado de precipitação no mês')
    ax[0].set_xlabel('Mês')
    ax[0].set_ylabel('Precipitação (mm)')
    ax[0].grid(True)
    
    ax[1].plot(meses, media_temperatura, marker='o')
    ax[1].set_title('Temperatura Média por mês')
    ax[1].set_xlabel('Mês')
    ax[1].set_ylabel('Temperatura (°C)')
    ax[1].grid(True)
    
    fig.tight_layout()
    
    return fig

In [ ]:
# mostrar grafico na tela
def mostrar_grafico(self):
    self.janelaGraficos = tk.Toplevel(self)
    self.janelaGraficos.title('Gráficos')
    
    self.janelaGraficos.protocol("WM_DELETE_WINDOW", self.janelaGraficos.destroy)
    
    # um dicionario com key = ano e value = caminho
    caminho = self.estacoes[self.estacaoSelecionada.get()]
    
    try:
        self.fig = get_plot(caminho)
        
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.janelaGraficos)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    except Exception as e:
        print(f'Erro: {e}')


## Respostas

Implemente sua aplicação como um pacote __Python__ e disponibilize neste __Notebook__ um explicação dos principais pontos da implementação. Utilize o comando mágico ``%run`` para executar o aplicativo desenvolvido.  Envie o __Notebook__ via __Moodle__ assim como o repositório com a implementação do aplicativo até o final do prazo. 